In [0]:
# ELMO is /was the state of art.. then came bert.. and then came gpt2..?
# elmo takes time  15 mins per ecpoch * 5 approx.. 

In [2]:
# Import le dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub # new from google
import os
import re
from keras import backend as K # stick to the conventions...
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

W0502 10:41:58.900794 139875380107136 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [3]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {} # empty dict
  data["sentence"] = [] #sentence is a list
  data["sentiment"] = [] # the corresponding sentiment list
  
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f: # file handle.. convoluted.. but works..
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos")) # as defined above
  neg_df = load_directory_data(os.path.join(directory, "neg")) # remember.. u use something more than 6 times.. make it a fn
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

  train_df = load_dataset(os.path.join(os.path.dirname(dataset),  # as defined above
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

  return train_df, test_df

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets() # and we use it right away
train_df.head()

,sentence,sentiment,polarity
0,Frustrated middle-aged Deputy District Attorne...,9,1
1,I saw this movie in my international cinema cl...,1,0
2,When I first started watching this movie I was...,2,0
3,"Actress Patty Duke wrote an insightful, funny,...",4,0
4,Children love dinosaurs. It's somewhat part of...,3,0


In [0]:
# Create datasets (Only take up to 150 words)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = train_df['polarity'].tolist()

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = test_df['polarity'].tolist()

In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [6]:
input_text = layers.Input(shape=(1,), dtype=tf.string) # layers is keras.layers # as imported above.. # input is 1 dimensional.. like 1 row.. columns..adjust as input comes in... type string
embedding = ElmoEmbeddingLayer()(input_text) 
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________


Not a very sophisticated NN is it..?

In [0]:
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=5,
          batch_size=32)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 955s 38ms/step - loss: 0.4581 - acc: 0.7812 - val_loss: 0.4178 - val_acc: 0.8038
Epoch 2/5
25000/25000 [==============================] - 953s 38ms/step - loss: 0.4060 - acc: 0.8108 - val_loss: 0.4149 - val_acc: 0.8072
Epoch 3/5
25000/25000 [==============================] - 957s 38ms/step - loss: 0.3883 - acc: 0.8233 - val_loss: 0.3876 - val_acc: 0.8240
Epoch 4/5
24992/25000 [============================>.] - ETA: 0s - loss: 0.3770 - acc: 0.8299

In [0]:
2+2